In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import ast
import re

In [ ]:
import numpy as np
import pandas as pd

path = '/content/drive/MyDrive/combined_reviews.xlsx'

#이모지 전처리된 데이터
#path = '/content/drive/MyDrive/processed_reviews.csv'

df = pd.read_excel(path)

In [ ]:


def clean_text(text):
    # 1. 비문자열이나 NaN 값이 있을 경우 빈 문자열로 처리
    if not isinstance(text, str):
        return ''

    # 2. 여러 공백을 하나의 공백으로 변경
    text = re.sub(r'\s+', ' ', text)  # 여러 공백을 하나로

    # 3. 텍스트 앞뒤 공백 제거
    text = text.strip()

    # 4. '.' 또는 '..'과 같이 마침표만 있는 텍스트를 빈 문자열로 처리
    if text in ['.', '..']:
        return ''



    return text



# 'Content' 컬럼에 정규화 적용
df['Content'] = df['Content'].apply(clean_text)
# 빈 문자열 제거
df = df[df['Content'] != '']

# 결과 출력
df[['Content']].head(20)


,Content
0,예전에 자주왔었는데 오랜만에 방문했습니다연어장덮밥 라지사이즈로 시켰는데 예전에 비해...
1,대창덮밥 진짜 살살 녹아요.. 연어장도 두툼하고 씹는 맛이 너무 좋아요!! 요리하시...
2,항상 맛잇어용 연어장도 하나 뺏어먹엇는데 맛잇네용🥹🥹
3,연어장동 맛있습니다심심하니
4,맛있어요
5,미나리 차슈라멘 맛나네영
6,스테키동 항상 맛잇어요이날은 뺏어먹은 대창덮밥도 맛잇네영
7,대창덥밥 좋아하는데 맛있게 먹었습니다~
8,고기가 많이 짜요 그리고 차슈라멘에 다시마(?)가 없어서 너무 아쉬웠어요. 하지만 ...
9,맛있었어요 근데 가게가 너무 지져분 . . 깔끔해지기전까진 안갈듯싶네요


In [ ]:
### (고민중) 형태소 분석과 문장 단순화 (고민중)

from konlpy.tag import Okt

okt = Okt()

def simplify_review(text):
    pos_tags = okt.pos(text)
    simplified_words = [word for word, tag in pos_tags if tag in ['Noun', 'Adjective', 'Verb']]
    return ' '.join(simplified_words)

df['Content'] = df['Content'].apply(simplify_review)

# 라벨링 !

In [ ]:
###최종 라벨링 코드

# 긍정/부정 키워드 정의
positive_keywords = ['좋', '맛', '최고', '추천', '행복', '만족', '친절', '훌륭', "깔끔", "감동", "맛집",'굿','굳','푸짐','마싯','많']
negative_keywords = ['별로', '맛없', '최악', '싫', '불친절', '아쉬움', '나쁨', '실망', '비려','딱딱']

# 긍정/부정 레이블 추가 함수
def assign_label(text):
    if any(keyword in text for keyword in positive_keywords):  # 긍정 키워드가 포함된 경우
        return 2
    elif any(keyword in text for keyword in negative_keywords):  # 부정 키워드가 포함된 경우
        return 1
    else:
        return 0  # 중립 또는 키워드가 없는 경우

# 레이블 추가
df['Label'] = df['Content'].apply(assign_label)


# 데이터를 Excel 파일로 저장
df.to_excel('/content/drive/MyDrive/combined_reviews.xlsx', index=False)


     Nickname                                            Content       Date  \
29       지으닝1                                                 쏘쏘   23.8.4.금   
32     jaywhy                                          서빙 꽃같이함ㅇㅇ  23.7.22.토   
44      큰아이70                       9500원 짜리가 제일 싼 곳이다. 큰맘먹고 간다.  23.4.22.토   
46        슬무드     대창덮밥 왜 시그니쳐인지 알겠어요 통통하고 매웠어요라멘은 계란 안줘서 좀 섭섭 ㅋㅋ  23.4.21.금   
47      박3918                                                 쩝쩝  23.4.17.월   
...       ...                                                ...        ...   
2068  Liz2113                    얌얌~벌써 쭈꾸미는 다 먹고볶음밥만 남았네요.얌얌~~^^    11.14.목   
2086    이소현77              맨날 너무 먹고싶어서 생각났다가 겨우 왔습니다! 다음에도 또 올게용    11.11.월   
2244  미식55555  밥이 질척거려서 이게 뭔지??테이블에 온갖 양념이 덕지덕지 묻어 있어서... 직접 ...    10.13.일   
2274      원치운                                역시 믿고 먹는 화리화리 입니다!!     10.7.월   
2361      운도옹                                               존밋 👍      9.6.금   

      Revisit Restaurant  리뷰길이  Label  
29         

형태소 분석 및 문장 단순화 후 koBERT 적용

In [ ]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 47.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 14.3 MB/s eta 0:00:00
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.20.0-py3-none-any.whl size=34818026 sha256=b3789c6b347e68a8006eb9d9deb05fc5697d97bebb789cbcbcf3465fe0940816
  Stored in directory: /root/.cache/pip/wheels/b6/b1/66/2be9840f8ef3627d63d93503d81a5e3b41e9498dcb63b00b13
Successfully built kiwipiepy-model


In [ ]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 26.2 MB/s eta 0:00:00


In [ ]:
#### 형태소 분석, 불용어 제거 등 전처리를 한 후, 감정분석 (실패..)

from kiwipiepy import Kiwi
from konlpy.tag import Okt
from transformers import BertTokenizer, BertForSequenceClassification
import torch
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# 형태소 분석기를 위한 준비
kiwi = Kiwi()
okt = Okt()

# 불용어 목록 로드
stopwords = pd.read_csv("https://raw.githubusercontent.com/yoonkt200/FastCampusDataset/master/korean_stopwords.txt", header=None).values.tolist()
stopwords = [word[0] for word in stopwords]  # 리스트로 변환

# 형태소 분석 및 단순화 함수
def simplify_text(text):
    # 입력이 문자열인지 확인하고, 그렇지 않으면 문자열로 변환
    if not isinstance(text, str):
        text = str(text)

    # 1. 형태소 분석 (Kiwi, Okt)
    # Kiwi로 분석 (명사, 동사, 형용사만 추출)
    tokens_kiwi = kiwi.analyze(text)
    tokens = [token[0] for token in tokens_kiwi[0][0] if token[1] in ['NNG', 'VV', 'VA']]  # Noun, Verb, Adjective

    # Okt로 분석하여 더 정확하게 명사/동사/형용사 추출
    tokens_okt = okt.pos(text)
    tokens.extend([word for word, tag in tokens_okt if tag in ['Noun', 'Verb', 'Adjective']])

    # 2. 불용어 제거
    tokens = [token for token in tokens if token not in stopwords]

    # 3. 기본형 변환 (동사, 형용사)
    # Okt를 사용한 기본형 변환
    tokens_okt_basic = [word if tag != 'Verb' and tag != 'Adjective' else okt.morphs(word)[0] for word, tag in okt.pos(text) if tag in ['Verb', 'Adjective', 'Noun']]

    # Kiwi를 사용한 기본형 변환
    tokens_kiwi_basic = [token[0] for token in tokens_kiwi[0][0] if token[1] in ['NNG', 'NNP', 'VV', 'VA']]

    # 4. 두 분석기의 결과를 통합하고 중복 제거
    final_tokens = list(set(tokens_okt_basic + tokens_kiwi_basic))

    # 최종 결과
    simplified_text = ' '.join(final_tokens)
    return simplified_text



# 리뷰 데이터 처리
df = pd.read_csv("/content/drive/MyDrive/combined_reviews.csv")  # 실제 데이터 경로로 수정
texts = df['Content'].tolist()  # df['Content']가 리뷰 텍스트 컬럼이라고 가정

# 텍스트 전처리
simplified_texts = [simplify_text(text) for text in texts]


# KoBERT 모델 로드
MODEL_NAME = 'beomi/kcbert-base'
tokenizer = BertTokenizer.from_pretrained(MODEL_NAME)
model = BertForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# 데이터셋 및 데이터로더
class ReviewDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len=128):
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        inputs = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )
        return {
            'input_ids': inputs['input_ids'].squeeze(0),
            'attention_mask': inputs['attention_mask'].squeeze(0)
        }

# 감정 분석 함수
def predict_sentiments(dataloader, model):
    results = []
    with torch.no_grad():
        for batch in tqdm(dataloader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask)
            probs = torch.softmax(outputs.logits, dim=1)  # 확률 계산
            results.extend(probs.cpu().numpy())  # 확률 값 리스트에 추가
    return results

# 단순화된 텍스트로 데이터셋 만들기
dataset = ReviewDataset(simplified_texts, tokenizer)
dataloader = DataLoader(dataset, batch_size=16)

# 감정 점수 계산
sentiment_scores = predict_sentiments(dataloader, model)

# 결과 출력 (긍정확률, 부정확률로 출력)
for text, score in zip(simplified_texts, sentiment_scores):
    positive_prob = score[1]  # 긍정 확률
    negative_prob = score[0]  # 부정 확률
    print(f"Content: {text}\nPositive Probability: {positive_prob:.4f}, Negative Probability: {negative_prob:.4f}\n")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/kcbert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 153/153 [00:16<00:00,  9.25it/s]


Content: 가격 자주 사장 양 친절하십니다 만 양은 라지 같습니다 밥 장 덮밥 비해 예전 하시고 연어 냅두 오 시켰는데 차라리 두는게 덮 줄 친절 냅 같 인상 사이즈 했습니다 것 왔었는데 좋 줄었네요 오랜만 라지사이즈로 좋았을 비하 방문
Positive Probability: 0.4000, Negative Probability: 0.6000

Content: 두툼 씹는 좋아요 추천 진짜 요리 합니다 장도 밥 솜씨 장 하시는 덮밥 연어 덮 녹 맛 친절 하 친절하고 분 직원 녹아요 좋 대창 살살
Positive Probability: 0.2686, Negative Probability: 0.7314

Content: 
Positive Probability: 0.6115, Negative Probability: 0.3885

Content: 잇네용 하나 용 뺏어 맛 먹 장도 잇어용 어장 연 는데 연어 항상
Positive Probability: 0.3690, Negative Probability: 0.6310

Content: 심심하 장동 심심하니 장 맛있 연어 맛있습니다
Positive Probability: 0.4746, Negative Probability: 0.5254

Content: 맛있어요 맛있
Positive Probability: 0.7340, Negative Probability: 0.2660

Content: 맛 영 라멘 차슈 나 미나리
Positive Probability: 0.8025, Negative Probability: 0.1975

Content: 맛잇네영 잇어요 덮 먹은 잇네 뺏어 맛 대창 영 잇 밥 스테키 날 스테키동 이날 덮밥 항상
Positive Probability: 0.5729, Negative Probability: 0.4271

Content: 대창덥밥 좋아하는데 먹었습니다 맛있게 덥 먹 좋 밥 맛있 대창
Positive Probability: 0.2930, Negative Probability: 0.7070

Co